In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install -U torch torchvision
!pip install git+https://github.com/facebookresearch/fvcore.git
import torch, torchvision
torch.__version__

Requirement already up-to-date: torch in /usr/local/lib/python3.6/dist-packages (1.3.1)
Requirement already up-to-date: torchvision in /usr/local/lib/python3.6/dist-packages (0.4.2)
  Cloning https://github.com/facebookresearch/fvcore.git to /tmp/pip-req-build-jc7pw5hc
  Running command git clone -q https://github.com/facebookresearch/fvcore.git /tmp/pip-req-build-jc7pw5hc
     |████████████████████████████████| 266kB 7.0MB/s 
  Created wheel for fvcore: filename=fvcore-0.1-cp36-none-any.whl size=50354 sha256=8022f6671c06664c16b7028ef6d5813e806abdde6ba5f6621349a16a00aaf40e
  Stored in directory: /tmp/pip-ephem-wheel-cache-g_e1xkda/wheels/48/53/79/3c6485543a4455a0006f5db590ab9957622b6227011941de06
  Created wheel for pyyaml: filename=PyYAML-5.2-cp36-cp36m-linux_x86_64.whl size=44209 sha256=f8130e6a47b6beec4a7ab85ccc482ed0966e9282f21ebf46b1bd19915baf0a16
  Stored in directory: /root/.cache/pip/wheels/54/b7/c7/2ada654ee54483c9329871665aaf4a6056c3ce36f29cf66e67
Successfully built fvcore py

'1.3.1'

In [0]:
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo

Cloning into 'detectron2_repo'...
remote: Enumerating objects: 1653, done.
remote: Total 1653 (delta 0), reused 0 (delta 0), pack-reused 1653
Receiving objects: 100% (1653/1653), 1.68 MiB | 27.23 MiB/s, done.
Resolving deltas: 100% (1024/1024), done.
Obtaining file:///content/detectron2_repo
     |████████████████████████████████| 2.1MB 4.8MB/s 
     |████████████████████████████████| 61kB 11.0MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: Pillow 4.3.0
    Uninstalling Pillow-4.3.0:
      Successfully uninstalled Pillow-4.3.0
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1
  Running setup.py develop for detectron2


In [0]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import matplotlib.pyplot as plt
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [0]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("just_nuts_3", {}, "/content/drive/My Drive/Colab Notebooks/cocodataset/train/trainval.json", "/content/drive/My Drive/Colab Notebooks/cocodataset/train/images")

In [0]:
just_nuts_metadata = MetadataCatalog.get("just_nuts_3")
dataset_dicts = DatasetCatalog.get("just_nuts_3")

WARNING [12/05 22:19:07 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[12/05 22:19:07 d2.data.datasets.coco]: Loaded 300 images in COCO format from /content/drive/My Drive/Colab Notebooks/cocodataset/train/trainval.json


In [0]:
import random

for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=just_nuts_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])

Output hidden; open in https://colab.research.google.com to view.

In [28]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import os

cfg = get_cfg()
cfg.merge_from_file("./detectron2_repo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_C4_3x.yaml")
cfg.DATASETS.TRAIN = ("just_nuts_2",)
cfg.DATASETS.TEST = ()   # no metrics implemented for this dataset
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_C4_3x/137849525/model_final_4ce675.pkl"  # initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.02
cfg.SOLVER.MAX_ITER = 300    # 300 iterations seems good enough, but you can certainly train longer
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3  # 3 classes (data, fig, hazelnut)

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

WARNING [12/05 23:12:37 d2.config.compat]: Config './detectron2_repo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_C4_3x.yaml' has no VERSION. Assuming it to be compatible with latest v2.
[12/05 23:12:37 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): ResNet(
    (stem): BasicStem(
      (conv1): Conv2d(
        3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
        (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
      )
    )
    (res2): Sequential(
      (0): BottleneckBlock(
        (shortcut): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
        )
        (conv1): Conv2d(
          64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv2): Conv2d(
          64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num

'roi_heads.box_predictor.cls_score.weight' has shape (81, 2048) in the checkpoint but (4, 2048) in the model! Skipped.
'roi_heads.box_predictor.cls_score.bias' has shape (81,) in the checkpoint but (4,) in the model! Skipped.
'roi_heads.box_predictor.bbox_pred.weight' has shape (320, 2048) in the checkpoint but (12, 2048) in the model! Skipped.
'roi_heads.box_predictor.bbox_pred.bias' has shape (320,) in the checkpoint but (12,) in the model! Skipped.
'roi_heads.mask_head.predictor.weight' has shape (80, 256, 1, 1) in the checkpoint but (3, 256, 1, 1) in the model! Skipped.
'roi_heads.mask_head.predictor.bias' has shape (80,) in the checkpoint but (3,) in the model! Skipped.


[12/05 23:12:38 d2.engine.train_loop]: Starting training from iteration 0
[12/05 23:12:47 d2.utils.events]: eta: 0:01:57  iter: 19  total_loss: 2.379  loss_cls: 0.813  loss_box_reg: 0.803  loss_mask: 0.650  loss_rpn_cls: 0.068  loss_rpn_loc: 0.008  time: 0.4154  data_time: 0.0299  lr: 0.000400  max_mem: 4407M
[12/05 23:12:55 d2.utils.events]: eta: 0:01:47  iter: 39  total_loss: 1.493  loss_cls: 0.323  loss_box_reg: 0.812  loss_mask: 0.315  loss_rpn_cls: 0.010  loss_rpn_loc: 0.006  time: 0.4085  data_time: 0.0055  lr: 0.000799  max_mem: 4407M
[12/05 23:13:03 d2.utils.events]: eta: 0:01:37  iter: 59  total_loss: 0.870  loss_cls: 0.102  loss_box_reg: 0.574  loss_mask: 0.152  loss_rpn_cls: 0.005  loss_rpn_loc: 0.007  time: 0.4068  data_time: 0.0066  lr: 0.001199  max_mem: 4407M
[12/05 23:13:11 d2.utils.events]: eta: 0:01:28  iter: 79  total_loss: 0.609  loss_cls: 0.064  loss_box_reg: 0.386  loss_mask: 0.125  loss_rpn_cls: 0.004  loss_rpn_loc: 0.006  time: 0.4050  data_time: 0.0123  lr: 0.0

OrderedDict()

In [0]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set the testing threshold for this model
cfg.DATASETS.TEST = ("just_nuts_3", )
predictor = DefaultPredictor(cfg)

In [0]:
from detectron2.utils.visualizer import ColorMode

for d in random.sample(dataset_dicts, 3):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=just_nuts_metadata, 
                   scale=0.8, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(v.get_image()[:, :, ::-1])

Output hidden; open in https://colab.research.google.com to view.

In [34]:
!pip install pytorch2keras